In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

import datetime 

### Extract Finance Data (Stock Price)

In [2]:
# Extract time series data of the stock price (Apple and S&P 500) from Yahoo Finance using yfinance library
apple= yf.Ticker("aapl")
sp = yf.Ticker("^GSPC")
applep = apple.history(start="2019-01-01", end="2022-12-31", interval="1wk")
spp = sp.history(start="2019-01-01", end="2022-12-31", interval="1wk")

In [3]:
columns_drop = ['Dividends', 'Stock Splits']
columns_drop_sp = ['Dividends', 'Volume', 'Stock Splits', 'Open', 'High', 'Low']
applep = applep.drop(columns=columns_drop)
spp = spp.drop(columns=columns_drop_sp)
applep = pd.merge(applep, spp, on="Date", how="left").reset_index()

In [4]:
applep

,Date,Open,High,Low,Close_x,Volume,Close_y
0,2019-01-01 00:00:00-05:00,37.317041,38.271110,34.211504,35.640194,966947200,2549.689941
1,2019-01-08 00:00:00-05:00,36.032904,37.230307,35.782343,36.138912,725470000,2582.610107
2,2019-01-15 00:00:00-05:00,36.203958,38.037405,36.150954,37.782024,491411200,2670.709961
3,2019-01-22 00:00:00-05:00,37.683252,38.097645,36.548488,37.656750,554774800,2643.850098
4,2019-01-29 00:00:00-05:00,37.644696,41.357367,37.129114,41.258587,830400800,2724.870117
...,...,...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,144.070019,150.689917,140.136039,146.406464,400668900,3998.840088
205,2022-12-06 00:00:00-05:00,146.845798,147.075443,139.786569,144.269730,343136300,3990.560059
206,2022-12-13 00:00:00-05:00,149.272088,149.741373,131.119811,132.168198,514858800,3817.659912
207,2022-12-20 00:00:00-05:00,131.189697,136.601432,129.442364,131.658981,305027800,3844.820068


### Extract Balance Sheet, Income Statement, and Financial Ratios data

In [5]:
# Extract finance data using Alpha Vintage library
import requests
import json

# set the API endpoint and parameters
symbol = 'AAPL'
api_key = 'U9NBHARV1LPZMLN0'

endpoint_earning = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={api_key}'
endpoint_ics = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
endpoint_bs = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}'

# send a request to the API endpoint
response_e = requests.get(endpoint_earning)
response_ics = requests.get(endpoint_ics)
response_bs = requests.get(endpoint_bs)

# parse the response JSON data
data_e = json.loads(response_e.content)
data_ics = json.loads(response_ics.content)
data_bs = json.loads(response_bs.content)

# cols needed 
cols = {
    'bs': ['cashAndCashEquivalentsAtCarryingValue', 'shortLongTermDebtTotal','commonStockSharesOutstanding', 'totalLiabilities', 'totalShareholderEquity'],
    'is': ['ebitda', 'netIncome'],
    'e' : ['reportedEPS']
    }


# extract the quarterly earnings data
earnings = data_e['quarterlyEarnings']
ics = data_ics['quarterlyReports']
bs = data_bs['quarterlyReports']

# convert the earnings data to a pandas dataframe
earnings_df = pd.DataFrame.from_dict(earnings)
ics_df = pd.DataFrame.from_dict(ics)
bs_df = pd.DataFrame.from_dict(bs)

# set the date column as the index and sort the dataframe by date
earnings_df = earnings_df.set_index('reportedDate').sort_index()
ics_df = ics_df.set_index('fiscalDateEnding').sort_index()
bs_df = bs_df.set_index('fiscalDateEnding').sort_index()

# filter the dataframe based on start and end dates
start_date = '2018-04-01'
end_date = '2023-03-27'

start_date_earning = '2019-01-29'
end_date_earning = '2022-10-27'

# filter specified columns from each df
earnings_df = earnings_df.loc[start_date_earning:end_date_earning][cols['e']]
ics_df = ics_df.loc[start_date:end_date][cols['is']]
bs_df = bs_df.loc[start_date:end_date][cols['bs']]

# Merge income statement and balance sheet dataframe 
df = ics_df.merge(bs_df, on='fiscalDateEnding')

# convert columns entries to float
for col in ['ebitda','commonStockSharesOutstanding', 'totalLiabilities', 'cashAndCashEquivalentsAtCarryingValue',
           'totalShareholderEquity', 'netIncome', 'shortLongTermDebtTotal']:
    df[col] = df[col].astype('float')
    
# calculate financial ratio
# Debt to Equity = total liabilities / total shareholder equity
# Return on Equity (ROE) = net Income / total shareholder equity
# Enterprise Value / EBITDA (ev/ebitda) = (oustanding shares + total debt - total cash) / EBITDA
df['debtToEquity'] = df['totalLiabilities'] / df['totalShareholderEquity']
df['roe'] = df['netIncome'] / df['totalShareholderEquity']
df['ev/ebitda'] = (df.commonStockSharesOutstanding + df.shortLongTermDebtTotal - df.cashAndCashEquivalentsAtCarryingValue) / df.ebitda

# add string date, year, quarter columns for each entries in BS+IS dataframe
temp_date = pd.Series(df.index).apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
temp_date.index = df.index
df['date'] = temp_date
df['year'] = df['date'].apply(lambda x: x.year)
df['quarter'] = df['date'].apply(lambda x: x.month / 3)

# add string date, year, quarter columns for each entries in earnings dataframe
temp_date1 = pd.Series(earnings_df.index).apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
temp_date1.index = earnings_df.index
earnings_df['date'] = temp_date1
earnings_df['year'] = earnings_df['date'].apply(lambda x: x.year)
earnings_df['quarter'] = earnings_df['date'].apply(lambda x: (x.month//3) +1)


In [6]:
df

,ebitda,netIncome,cashAndCashEquivalentsAtCarryingValue,shortLongTermDebtTotal,commonStockSharesOutstanding,totalLiabilities,totalShareholderEquity,debtToEquity,roe,ev/ebitda,date,year,quarter
fiscalDateEnding,,,,,,,,,,,,,
2018-06-30,1.679500e+10,1.151900e+10,3.197100e+10,1.158710e+11,4.842917e+09,2.342480e+11,1.149490e+11,2.037843,0.100210,5.283889,2018-06-30,2018,2.0
2018-09-30,2.004300e+10,1.412500e+10,2.591300e+10,2.201320e+11,4.754986e+09,2.585780e+11,1.071470e+11,2.413301,0.131828,9.927355,2018-09-30,2018,3.0
2018-12-31,2.819100e+10,1.996500e+10,4.477100e+10,1.155290e+11,4.729803e+09,2.558270e+11,1.178920e+11,2.170012,0.169350,2.677727,2018-12-31,2018,4.0
2019-03-31,1.784300e+10,1.156100e+10,3.798800e+10,1.129220e+11,4.607284e+09,2.361380e+11,1.058600e+11,2.230663,0.109210,4.457843,2019-03-31,2019,1.0
2019-06-30,1.571000e+10,1.004400e+10,5.053000e+10,1.080590e+11,4.531395e+09,2.257830e+11,9.645600e+10,2.340788,0.104130,3.950375,2019-06-30,2019,2.0
2019-09-30,2.011600e+10,1.368600e+10,4.884400e+10,2.091350e+11,4.443236e+09,2.480280e+11,9.048800e+10,2.741004,0.151247,8.189214,2019-09-30,2019,3.0
2019-12-31,2.951900e+10,2.223600e+10,3.977100e+10,1.078610e+11,4.384959e+09,2.510870e+11,8.953100e+10,2.804470,0.248361,2.455197,2019-12-31,2019,4.0
2020-03-31,1.667800e+10,1.124900e+10,4.017400e+10,1.077770e+11,4.323987e+09,2.419750e+11,7.842500e+10,3.085432,0.143436,4.312687,2020-03-31,2020,1.0
2020-06-30,1.658600e+10,1.125300e+10,3.338300e+10,1.109870e+11,4.283939e+09,2.450620e+11,7.228200e+10,3.390360,0.155682,4.937172,2020-06-30,2020,2.0


In [7]:
earnings_df

,reportedEPS,date,year,quarter
reportedDate,,,,
2019-01-29,1.05,2019-01-29,2019,1
2019-04-30,0.62,2019-04-30,2019,2
2019-07-30,0.55,2019-07-30,2019,3
2019-10-30,0.76,2019-10-30,2019,4
2020-01-28,1.25,2020-01-28,2020,1
2020-04-30,0.64,2020-04-30,2020,2
2020-07-30,0.65,2020-07-30,2020,3
2020-10-29,0.73,2020-10-29,2020,4
2021-01-27,1.68,2021-01-27,2021,1


### Extract US GDP and Inflation data

In [8]:
# Extract US GDP from Federal Reserve Economic Data (FRED)
url_gdp_us = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=GDP&scale=left&cosd=1947-01-01&coed=2022-10-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2023-02-20&revision_date=2023-02-20&nd=1947-01-01"
gdp_us = pd.read_csv(url_gdp_us)

gdp = gdp_us.loc[::-1].reset_index().drop(columns='index').rename(columns={'DATE':'period', 'GDP':'gdp'})


In [9]:
# Add year and quarter columns to the dataframe
gdp['year'] = gdp['period'].str[:4]
gdp['year'] = gdp['year'].astype(int)

gdp['quarter'] = gdp['period'].str[5:7]

quarter_dict = {
    '01': 1,
    '04': 2,
    '07': 3,
    '10': 4
}

gdp['quarter'] = gdp['quarter'].apply(lambda x: quarter_dict[x])

In [10]:
gdp

,period,gdp,year,quarter
0,2022-10-01,26137.992,2022,4
1,2022-07-01,25723.941,2022,3
2,2022-04-01,25248.476,2022,2
3,2022-01-01,24740.480,2022,1
4,2021-10-01,24349.121,2021,4
...,...,...,...,...
299,1948-01-01,265.742,1948,1
300,1947-10-01,259.745,1947,4
301,1947-07-01,249.585,1947,3
302,1947-04-01,245.968,1947,2


In [11]:
# Extract CPI data from US Bureau of Labor Statistics (BLS)
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0'],"startyear":"2014", "endyear":"2023"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

In [12]:
cpi_us_month = pd.DataFrame(json_data['Results']['series'][0]['data'])
cpi_us_month = cpi_us_month.rename(columns={'value':'cpi'})
cpi_us_month.drop(columns=['latest','footnotes'], inplace=True)

In [13]:
cpi_us_month['period'] = cpi_us_month['period'].str[1:]
cpi_us_month = cpi_us_month.astype({'period':'int32'})

In [14]:
#find the the latest quantile month index, datasets provide monthly CPI

first_12 = 0
for index, row in cpi_us_month.iterrows():
  if (row.period == 12):
    first = index
    break

In [15]:
cpi_us_month = cpi_us_month.loc[first_12+1:]
cpi_us_month = cpi_us_month.astype({'year': str, 'period': str})
cpi_us_month['year']  = cpi_us_month['year'] + "-" 
cpi_us_month['period'] = cpi_us_month['year'] + cpi_us_month['period'] 
cpi = cpi_us_month.drop(columns=['periodName', 'year'])

In [16]:
cpi = cpi.iloc[len(cpi):0:-1].reset_index()

# calculate Inflation rate
cpi['cpi'] = cpi['cpi'].astype('float')
cpi.loc[0, 'infl'] = 0
for index, row in cpi.loc[1:].iterrows():
    cpi.loc[index, 'infl'] = (cpi.loc[index, 'cpi'] - cpi.loc[index -1, 'cpi']) * 100 / cpi.loc[index -1, 'cpi']

# add year and quarter columns to the dataframe
cpi['year'] = cpi['period'].str[:4]
cpi['year'] = cpi['year'].astype(int)

cpi['month'] = cpi['period'].str[5:7]
cpi['month'] = cpi['month'].astype(int)

In [17]:
cpi

,index,period,cpi,infl,year,month
0,109,2014-1,233.916,0.000000,2014,1
1,108,2014-2,234.781,0.369791,2014,2
2,107,2014-3,236.293,0.644004,2014,3
3,106,2014-4,237.072,0.329675,2014,4
4,105,2014-5,237.900,0.349261,2014,5
...,...,...,...,...,...,...
103,6,2022-8,296.171,-0.035440,2022,8
104,5,2022-9,296.808,0.215078,2022,9
105,4,2022-10,298.012,0.405649,2022,10
106,3,2022-11,297.711,-0.101003,2022,11


### Combine Daily Stock Price and Periodic Data (Financial Ratio and Macroeconomic Data)

In [18]:
# While stock price data is daily, Financial ratio and macro data are monthly or quarterly
# Find the respective financial ratio and macro data according to the date, matched by quarter or month of which
# the daily data is in 
def getData(date, var ,df):
    quarter = (int(date.month - 1) // 3) + 1
    year = date.year
    return df[(df.quarter == quarter) & (df.year == year)][var].values

def getData_month(date, var ,df):
    month = date.month
    year = date.year
    return df[(df.month == month) & (df.year == year)][var].values

In [19]:
applep_new = applep.copy()
applep_new['roe'] = applep_new['Date'].apply(lambda x: getData(x,'roe', df)[0])
applep_new['debtToEquity'] = applep_new['Date'].apply(lambda x: getData(x,'debtToEquity', df)[0])
applep_new['ev/ebitda']  = applep_new['Date'].apply(lambda x: getData(x,'ev/ebitda', df)[0])
applep_new['eps'] = applep_new['Date'].apply(lambda x: getData(x,'reportedEPS', earnings_df)[0])
applep_new['gdp'] = applep_new['Date'].apply(lambda x: getData(x,'gdp', gdp)[0])
applep_new['infl'] = applep_new['Date'].apply(lambda x: getData_month(x,'infl', cpi)[0])
applep_new['dateStr'] = applep_new['Date'].dt.strftime('%Y-%m-%d')

In [20]:
applep_new

,Date,Open,High,Low,Close_x,Volume,Close_y,roe,debtToEquity,ev/ebitda,eps,gdp,infl,dateStr
0,2019-01-01 00:00:00-05:00,37.317041,38.271110,34.211504,35.640194,966947200,2549.689941,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-01
1,2019-01-08 00:00:00-05:00,36.032904,37.230307,35.782343,36.138912,725470000,2582.610107,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-08
2,2019-01-15 00:00:00-05:00,36.203958,38.037405,36.150954,37.782024,491411200,2670.709961,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-15
3,2019-01-22 00:00:00-05:00,37.683252,38.097645,36.548488,37.656750,554774800,2643.850098,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-22
4,2019-01-29 00:00:00-05:00,37.644696,41.357367,37.129114,41.258587,830400800,2724.870117,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,144.070019,150.689917,140.136039,146.406464,400668900,3998.840088,0.528813,5.112557,2.692085,1.29,26137.992,-0.101003,2022-11-29
205,2022-12-06 00:00:00-05:00,146.845798,147.075443,139.786569,144.269730,343136300,3990.560059,0.528813,5.112557,2.692085,1.29,26137.992,-0.307009,2022-12-06
206,2022-12-13 00:00:00-05:00,149.272088,149.741373,131.119811,132.168198,514858800,3817.659912,0.528813,5.112557,2.692085,1.29,26137.992,-0.307009,2022-12-13
207,2022-12-20 00:00:00-05:00,131.189697,136.601432,129.442364,131.658981,305027800,3844.820068,0.528813,5.112557,2.692085,1.29,26137.992,-0.307009,2022-12-20


### Combine Data with Twitter and Reddit Sentiment Score

In [21]:
twitter = pd.read_csv('twitterComments.csv')

In [22]:
reddit = pd.read_csv('reddit sentiment scores.csv')

In [23]:
applep_new = applep_new.merge(reddit[['Week Start', 'Weighted Sentiment Score']], how='left',
                                 left_on='dateStr', right_on='Week Start')
applep_new.drop('Week Start', axis=1, inplace=True)
applep_new['Weighted Sentiment Score'] = applep_new['Weighted Sentiment Score'].fillna(0)
applep_new = applep_new.rename(columns={'Weighted Sentiment Score': 'reddit_sentiment'})

In [24]:
applep_new = applep_new.merge(twitter[['Week Start', 'Twitter_Weighted_Sentiment_Score' ]], how='left', 
                                    left_on='dateStr',right_on='Week Start')
applep_new.drop('Week Start', axis=1, inplace=True)
applep_new = applep_new.rename(columns={'Twitter_Weighted_Sentiment_Score': 'twitter_sentiment'})

In [25]:
applep_new

,Date,Open,High,Low,Close_x,Volume,Close_y,roe,debtToEquity,ev/ebitda,eps,gdp,infl,dateStr,reddit_sentiment,twitter_sentiment
0,2019-01-01 00:00:00-05:00,37.317041,38.271110,34.211504,35.640194,966947200,2549.689941,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-01,0.000000,0.048722
1,2019-01-08 00:00:00-05:00,36.032904,37.230307,35.782343,36.138912,725470000,2582.610107,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-08,0.000000,0.175603
2,2019-01-15 00:00:00-05:00,36.203958,38.037405,36.150954,37.782024,491411200,2670.709961,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-15,0.000000,0.225401
3,2019-01-22 00:00:00-05:00,37.683252,38.097645,36.548488,37.656750,554774800,2643.850098,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-22,0.000000,0.153582
4,2019-01-29 00:00:00-05:00,37.644696,41.357367,37.129114,41.258587,830400800,2724.870117,0.109210,2.230663,4.457843,1.05,21013.085,0.190660,2019-01-29,0.000000,0.127374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,144.070019,150.689917,140.136039,146.406464,400668900,3998.840088,0.528813,5.112557,2.692085,1.29,26137.992,-0.101003,2022-11-29,0.152096,0.068209
205,2022-12-06 00:00:00-05:00,146.845798,147.075443,139.786569,144.269730,343136300,3990.560059,0.528813,5.112557,2.692085,1.29,26137.992,-0.307009,2022-12-06,0.158785,0.222123
206,2022-12-13 00:00:00-05:00,149.272088,149.741373,131.119811,132.168198,514858800,3817.659912,0.528813,5.112557,2.692085,1.29,26137.992,-0.307009,2022-12-13,0.000000,0.105450
207,2022-12-20 00:00:00-05:00,131.189697,136.601432,129.442364,131.658981,305027800,3844.820068,0.528813,5.112557,2.692085,1.29,26137.992,-0.307009,2022-12-20,0.000000,0.080198


In [26]:
applep_new.columns

Index(['Date', 'Open', 'High', 'Low', 'Close_x', 'Volume', 'Close_y', 'roe',
       'debtToEquity', 'ev/ebitda', 'eps', 'gdp', 'infl', 'dateStr',
       'reddit_sentiment', 'twitter_sentiment'],
      dtype='object')

In [27]:
applep_new.to_csv('data.csv')